In [1]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

Mapping the thresholded tracts from Omuircheartaigh and Jbabdi 2018 to the Talairach atlas

Goals:
1 Qualify the regions from the Author's ICA with respect to standard regions of interest found in the Talairach atlas.
2 Quantify how many voxels from the ICAs are in each of the represented standard ROIs.
3 Quantify the proportion of the ROI that is represented by the ICA of interest.

In [2]:
Talairach_path = '/usr/local/fsl/data/atlases/Talairach/Talairach-labels-2mm.nii.gz'
Omuir_path = '/home/corey/atlases/Omuir2018/SupplementaryImages/SI-9-ICsTracts050_MM_2mm.nii.gz' 

In [3]:
#The difficult part: Establishing which labels of the Talairach atlas are encompassed by the non-zero values of each white matter tract (from ICA)

#Create array of values and call unique function to return those encompassed

Tal_atlas = nib.load(Talairach_path)
Tal_data = Tal_atlas.get_fdata()
u_labels = np.unique(Tal_data,return_counts=True)

In [5]:
np.set_printoptions(suppress=True)

In [7]:
1 == u_labels[0][1]

True

#Isolate voxels that are non-zero in each Omuircheartaigh tract, qualify unique Talairach regions, quantify proportions

In [9]:
#Load Omuircheartaigh image:
omuir_image = nib.load(Omuir_path)
omuir_data = omuir_image.get_fdata()

In [11]:
#Number of non-zero value voxels for each Omuircheartaigh ICA
omuir_nonzero_counts = []
for v in range(0,omuir_data.shape[3]):
    count = np.count_nonzero(omuir_data[:,:,:,v])
    omuir_nonzero_counts.append(count)

In [13]:
tal_flags = []
tal_flag_counts = []
ICA_proportions = []
for v in range(0,omuir_data.shape[3]):
    tal_flag = []
    for x in range(0,omuir_data.shape[0]):
        for y in range(0, omuir_data.shape[1]):
            for z in range(0,omuir_data.shape[2]):
                if omuir_data[x,y,z,v] != 0 and Tal_data[x,y,z] != 0:
                    tal_flag.append(Tal_data[x,y,z])
    t_flag = np.unique(tal_flag,return_counts=True)
    tal_flags.append(t_flag[0])
    tal_flag_counts.append(t_flag[1])
    
    IC_prop = t_flag[1] / omuir_nonzero_counts[v]
    ICA_proportions.append(IC_prop)


Since there are many (sometimes hundreds) of ROIs in each ICA, and the majority of the ROIs in each ICA constitute a small proportion of the ICA (often < 1%), we should sort by the ROIs that comprise the largest proportions of the ICAs and that make up no less than 10% of the ICA

In [17]:
coupled_flags_and_proportions = [tal_flags,ICA_proportions]

In [20]:
cfp = tuple(coupled_flags_and_proportions)

In [41]:
sorted_flags = []
sorted_props = []
for v in range(0,50):
    tal_flag_vlist = tal_flags[v].tolist()
    ic_prop_vlist = ICA_proportions[v].tolist()

    list1, list2 = zip(*sorted(zip(ic_prop_vlist,tal_flag_vlist),reverse=True))
    sorted_flags.append(list2)
    sorted_props.append(list1)

In [42]:
print(sorted_flags[48][0:3])
print(sorted_props[48])

(39.0, 24.0, 51.0)
(0.10910698496905394, 0.09024462127910404, 0.06224580017683466, 0.04921898025346301, 0.04385499557913351, 0.03660477453580902, 0.026348364279398764, 0.021750663129973476, 0.019982316534040673, 0.019805481874447393, 0.018626584143825523, 0.016091954022988506, 0.01479516651930445, 0.013616268788682582, 0.012614205717653993, 0.011081638667845565, 0.009666961391099321, 0.00842911877394636, 0.007014441497200118, 0.006483937518420277, 0.005658709106984969, 0.005481874447391689, 0.005481874447391689, 0.0050103153551429415, 0.004244031830238726, 0.004185086943707633, 0.00412614205717654, 0.004067197170645446, 0.0032419687592101384, 0.0031240789861479517, 0.0030651340996168583, 0.0030651340996168583, 0.003006189213085765, 0.002770409666961391, 0.002652519893899204, 0.0025935750073681108, 0.002475685234305924, 0.0024167403477748303, 0.0024167403477748303, 0.0024167403477748303, 0.0022988505747126436, 0.0022988505747126436, 0.002180960801650457, 0.002122015915119363, 0.00212201

In [47]:
TOP_flags_and_props = []
only_flags = []
for v in range(0,50):
    top_flags = sorted_flags[v][0:3]
    top_props = sorted_props[v][0:3]
    top_together = (top_flags,top_props)
    TOP_flags_and_props.append(top_together)
    only_flags.append(top_flags)

In [48]:
only_flags

[(133.0, 173.0, 51.0),
 (485.0, 88.0, 51.0),
 (347.0, 51.0, 485.0),
 (879.0, 878.0, 784.0),
 (195.0, 193.0, 155.0),
 (51.0, 157.0, 252.0),
 (4.0, 687.0, 194.0),
 (796.0, 879.0, 878.0),
 (193.0, 194.0, 195.0),
 (24.0, 51.0, 39.0),
 (309.0, 52.0, 486.0),
 (798.0, 194.0, 880.0),
 (860.0, 864.0, 968.0),
 (798.0, 786.0, 881.0),
 (347.0, 439.0, 39.0),
 (193.0, 347.0, 195.0),
 (134.0, 174.0, 664.0),
 (194.0, 687.0, 196.0),
 (685.0, 193.0, 784.0),
 (194.0, 196.0, 28.0),
 (922.0, 972.0, 107.0),
 (194.0, 348.0, 156.0),
 (230.0, 231.0, 107.0),
 (348.0, 231.0, 156.0),
 (193.0, 194.0, 687.0),
 (193.0, 685.0, 682.0),
 (28.0, 42.0, 52.0),
 (348.0, 4.0, 89.0),
 (89.0, 486.0, 253.0),
 (42.0, 52.0, 28.0),
 (309.0, 52.0, 158.0),
 (52.0, 253.0, 28.0),
 (860.0, 51.0, 308.0),
 (196.0, 156.0, 194.0),
 (921.0, 230.0, 860.0),
 (348.0, 190.0, 194.0),
 (195.0, 193.0, 51.0),
 (798.0, 879.0, 796.0),
 (193.0, 347.0, 155.0),
 (193.0, 194.0, 347.0),
 (864.0, 52.0, 309.0),
 (51.0, 308.0, 485.0),
 (189.0, 347.0, 193.0)

In [44]:
TOP_flags_and_props[0][0]

(133.0, 173.0, 51.0)

Automate reading of html file mapping Talairach labels to region names

Calculate proportion of each Talairach region represented by voxels from the ICA where it spans that region